In [ ]:
import pandas as pd
import numpy as np
import scipy.linalg as linalg
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as mtick
import array_to_latex as a2l
import seaborn as sns
import math
from matplotlib.pylab import figure, subplot, plot, xlabel, ylabel, hist, show
import sklearn.linear_model as lm
from sklearn import model_selection
from toolbox_02450 import rlr_validate


df = pd.read_csv("../Prostate_Cancer.csv", index_col="id")
df["diagnosis_result"] = np.where(df["diagnosis_result"] == "M", 1, 0)

continuous_cols = ['radius', 
                    'texture', 
                    'perimeter', 
                    'area', 
                    'smoothness', 
                    'compactness', 
                    'symmetry', 
                    'fractal_dimension']
                    
df_means = df[continuous_cols].mean()

df_std = df[continuous_cols].std()

df[continuous_cols] = (df[continuous_cols] -  df_means) / df_std

X = df.to_numpy() # df as a numpy array

/home/piripuz/Università/Magistrale/mach_learning/Progetto/Intro_To_ML/Project2/toolbox_02450/__init__.py:115: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if display is 'verbose':


# Part A

In [ ]:
# Field selected for regression:
target_column = "smoothness"

continuous_cols.remove(target_column)

y = df[target_column]
X = df[continuous_cols]

X = X.to_numpy()
y = y.to_numpy()

N, M = X.shape

In [ ]:
K = 5
CV = model_selection.KFold(K, shuffle=True, random_state=100)

lambdas = np.logspace(1.2, 3, num=20)

Error_train = np.empty((K,1))
Error_test = np.empty((K,1))
Error_train_rlr = np.empty((K,1))
Error_test_rlr = np.empty((K,1))
Error_train_nofeatures = np.empty((K,1))
Error_test_nofeatures = np.empty((K,1))
opt_lambdas = np.empty((K,1))
error = np.empty((K,1))

w_rlr = np.empty((M,K))
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
w_noreg = np.empty((M,K))

In [ ]:

model = lm.LinearRegression()

In [ ]:
k=0
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10

    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)

    error[k] = opt_val_err
    opt_lambdas[k] = opt_lambda
    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :] 
    
    Xty = X_train.T @ y_train
    XtX = X_train.T @ X_train

    Error_train_nofeatures[k] = np.square(y_train-y_train.mean()).sum(axis=0)/y_train.shape[0]
    Error_test_nofeatures[k] = np.square(y_test-y_test.mean()).sum(axis=0)/y_test.shape[0]
    
    # Compute mean squared error without using the input data at all
    Error_train_nofeatures[k] = np.square(y_train-y_train.mean()).sum(axis=0)/y_train.shape[0]
    Error_test_nofeatures[k] = np.square(y_test-y_test.mean()).sum(axis=0)/y_test.shape[0]

    # Estimate weights for the optimal value of lambda, on entire training set
    lambdaI = opt_lambda * np.eye(M)
    lambdaI[0,0] = 0 # Do no regularize the bias term
    w_rlr[:,k] = np.linalg.solve(XtX+lambdaI,Xty).squeeze()
    # Compute mean squared error with regularization with optimal lambda
    Error_train_rlr[k] = np.square(y_train-X_train @ w_rlr[:,k]).sum(axis=0)/y_train.shape[0]
    Error_test_rlr[k] = np.square(y_test-X_test @ w_rlr[:,k]).sum(axis=0)/y_test.shape[0]
    Error_train_nofeatures = np.empty((K,1))
    Error_test_nofeatures = np.empty((K,1))

    # OR ALTERNATIVELY: you can use sklearn.linear_model module for linear regression:
    model.fit(X_train, y_train)
    Error_train[k] = np.square(y_train-model.predict(X_train)).sum()/y_train.shape[0]
    Error_test[k] = np.square(y_test-model.predict(X_test)).sum()/y_test.shape[0]

    # Display the results for the last cross-validation fold
    if k == K-1:
        figure(k, figsize=(13,8))
        subplot(1,2,1)
        semilogx(lambdas,mean_w_vs_lambda.T[:,1:],'.-') # Don't plot the bias term
        xlabel('Regularization factor')
        ylabel('Mean Coefficient Values')
        grid()
        # You can choose to display the legend, but it's omitted for a cleaner 
        # plot, since there are many attributes
        #legend(attributeNames[1:], loc='best')
        
        subplot(1,2,2)
        title('Optimal lambda: {0}'.format(np.format_float_scientific(opt_lambda, precision=2)))
        loglog(lambdas,train_err_vs_lambda.T,'b.-',lambdas,test_err_vs_lambda.T,'r.-')
        xlabel('Regularization factor')
        ylabel('Squared error (crossvalidation)')
        legend(['Train error','Validation error'])
        grid()
    
    # To inspect the used indices, use these print statements
    #print('Cross validation fold {0}/{1}:'.format(k+1,K))
    #print('Train indices: {0}'.format(train_index))
    #print('Test indices: {0}\n'.format(test_index))

    k+=1

   

In [ ]:
print('Linear regression without feature selection:')
print('- Training error: {0}'.format(Error_train.mean()))
print('- Test error:     {0}'.format(Error_test.mean()))
print('- R^2 train:     {0}'.format((Error_train_nofeatures.sum()-Error_train.sum())/Error_train_nofeatures.sum()))
print('- R^2 test:     {0}\n'.format((Error_test_nofeatures.sum()-Error_test.sum())/Error_test_nofeatures.sum()))
print('Regularized linear regression:')
print('- Training error: {0}'.format(Error_train_rlr.mean()))
print('- Test error:     {0}'.format(Error_test_rlr.mean()))
print('- R^2 train:     {0}'.format((Error_train_nofeatures.sum()-Error_train_rlr.sum())/Error_train_nofeatures.sum()))
print('- R^2 test:     {0}\n'.format((Error_test_nofeatures.sum()-Error_test_rlr.sum())/Error_test_nofeatures.sum()))

print('Weights in last fold:')
for m in range(M):
    print('{:>15} {:>15}'.format(continuous_cols[m], np.round(w_rlr[m,-1],2)))


# Part B

In [ ]:
from sklearn.neural_network import MLPRegressor

In [11]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.neural_network import MLPRegressor

# Define your dataset (X and y)

# Define the number of folds
K1 = 10
K2 = 10

# Define the range of values for h (number of hidden units) and λ (regularization parameter)
hidden_units = [1, 5, 10, 20]  # Example values for h
lambda_values = [0.001, 0.01, 0.1, 1.0]  # Example values for λ

# Initialize lists to store the results
regression_scores = []  # For regularized linear regression
ann_scores = []  # For artificial neural network
baseline_scores = []  # For the baseline model

# Define two-level cross-validation
outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)

for train_idx, test_idx in outer_cv.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # First, define a baseline model that predicts the mean of y
    baseline_model = DummyRegressor(strategy="mean")
    baseline_model.fit(X_train, y_train)
    baseline_pred = baseline_model.predict(X_test)
    baseline_score = mean_squared_error(y_test, baseline_pred)
    baseline_scores.append(baseline_score)

    # Second, implement two-level cross-validation for the ANN model
    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)

    best_ann_score = float('inf')
    best_h = None
    best_lambda = None

    # Loop through all hidden unit values and lambda values and perform KFold to find the best values
    for h in hidden_units:
        for lambda_value in lambda_values:
            ann_scores_inner = []

            for train_inner_idx, val_idx in inner_cv.split(X_train, y_train):
                X_train_inner, X_val = X_train[train_inner_idx], X_train[val_idx]
                y_train_inner, y_val = y_train[train_inner_idx], y_train[val_idx]

                # Build the ANN model with h hidden units and λ regularization
                ann_model = MLPRegressor(hidden_layer_sizes=(h,), alpha=lambda_value, max_iter=200, batch_size=10, learning_rate_init=0.001)
                ann_model.fit(X_train_inner, y_train_inner)

                # Evaluate the ANN model
                ann_pred = ann_model.predict(X_val)
                ann_score = mean_squared_error(y_val, ann_pred)
                ann_scores_inner.append(ann_score)

            mean_ann_score = np.mean(ann_scores_inner)

            # Check if previous KFold found best results
            if mean_ann_score < best_ann_score:
                best_ann_score = mean_ann_score
                best_h = h
                best_lambda = lambda_value
    print("K")
    print(best_lambda)
    print(best_ann_score)

    # Train the ANN model on the entire training set with the best hyperparameters
    final_ann_model = MLPRegressor(hidden_layer_sizes=(best_h,), alpha=best_lambda, max_iter=200, batch_size=10, learning_rate_init=0.001)
    final_ann_model.fit(X_train, y_train)

    # Evaluate the final ANN model
    ann_pred_final = final_ann_model.predict(X_test)
    ann_score_final = mean_squared_error(y_test, ann_pred_final)
    ann_scores.append(ann_score_final)

    # Train and evaluate the regularized linear regression model
    ridge_model = Ridge(alpha=best_lambda)
    ridge_model.fit(X_train, y_train)
    ridge_pred = ridge_model.predict(X_test)
    ridge_score = mean_squared_error(y_test, ridge_pred)
    regression_scores.append(ridge_score)

# Compare the results
print("Baseline Scores:", baseline_scores)
print("ANN Scores:", ann_scores)
print("Linear Regression Scores:", regression_scores)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

Baseline Scores: [1.3774288284187821, 1.212629282167077, 0.6061283452432312, 0.554760218642264, 0.6413581267784418, 1.4118230140829398, 0.7994498710273665, 0.7574109333382335, 1.699673410330029, 1.0033542674678464]
ANN Scores: [0.6612626118317203, 1.2481651225569483, 0.5223145324582984, 0.4532145232049423, 0.29157922642408785, 1.2967979473688496, 0.9318969552642811, 0.39361578909586104, 1.9299915037344793, 0.8790951696920377]
Linear Regression Scores: [0.825670277666398, 1.2530628295677677, 0.503808582185972, 0.5069492834036179, 0.35690393227627376, 2.0731804482805862, 0.8548237802127854, 0.5503603251203197, 1.5713942793215712, 0.7061005261316904]


In [13]:
# Compare the results


print("Baseline Scores:", np.array(baseline_scores).mean())
print("ANN Scores:", np.array(ann_scores).mean())
print("Linear Regression Scores:", np.array(regression_scores).mean())

Baseline Scores: 1.0064016297496212
ANN Scores: 0.8607933381631506
Linear Regression Scores: 0.9202254264166984
